# Pré-Processo (2)

Criando novas variáveis

## Leitura inicial

Definir SparkContext e SQLContext

In [32]:
#from pyspark import SparkContext
#sc = SparkContext()
#from pyspark.sql import SQLContext
#sqlContext=SQLContext(sc)
#import pandas as pd

Lendo o arquivo de trabalho

In [54]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).load("file:/home/cloudera/Time.csv", inferSchema=True
)

In [55]:
bd=bd.select('Year','Month','DAY_OF_MONTH','DAY_OF_WEEK','CRS_DEP_TIME','OP_UNIQUE_CARRIER',
             'TAIL_NUM', 'ARR_DELAY','DEP_DELAY','ORIGIN','DEST','DISTANCE','CANCELLED', 'DIVERTED',
             'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY')

In [56]:
bd2 = bd.filter( bd.ORIGIN.isin('ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') & bd.DEST.isin(
    'ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') )

sqlContext.registerDataFrameAsTable(bd2, "bd2")

In [57]:
bd3 = bd2.filter(bd2.CANCELLED==0
).filter(bd2.DIVERTED==0)

In [58]:
bd4 = bd3.na.fill({'CARRIER_DELAY':0, 'WEATHER_DELAY':0,'NAS_DELAY':0,'SECURITY_DELAY':0, 'LATE_AIRCRAFT_DELAY':0})
#bd4 = bd3.na.fill(0)
#bd4 = bd3.na.drop()

## Transformação e criação de novas variáveis

In [59]:
from pyspark.sql.functions import log

bd4 = bd4.withColumn('LogD', log(10.0, 'Distance'))

In [60]:
bd4 = bd4.withColumn('DELAY', (bd4.ARR_DELAY >=15).cast('int'))

In [63]:
bd4 = bd4.withColumn('RETARD', bd4.ARR_DELAY-bd4.DEP_DELAY)

In [64]:
bd4.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('DAY_OF_MONTH', 'int'),
 ('DAY_OF_WEEK', 'int'),
 ('CRS_DEP_TIME', 'int'),
 ('OP_UNIQUE_CARRIER', 'string'),
 ('TAIL_NUM', 'string'),
 ('ARR_DELAY', 'double'),
 ('DEP_DELAY', 'double'),
 ('ORIGIN', 'string'),
 ('DEST', 'string'),
 ('DISTANCE', 'double'),
 ('CANCELLED', 'double'),
 ('DIVERTED', 'double'),
 ('CARRIER_DELAY', 'double'),
 ('WEATHER_DELAY', 'double'),
 ('NAS_DELAY', 'double'),
 ('SECURITY_DELAY', 'double'),
 ('LATE_AIRCRAFT_DELAY', 'double'),
 ('LogD', 'double'),
 ('DELAY', 'int'),
 ('DETARD', 'double'),
 ('RETARD', 'double')]

In [65]:
bd4.describe('ARR_DELAY','LogD','DELAY','RETARD').show()

+-------+------------------+------------------+------------------+------------------+
|summary|         ARR_DELAY|              LogD|             DELAY|            RETARD|
+-------+------------------+------------------+------------------+------------------+
|  count|             30466|             30466|             30466|             30466|
|   mean|13.300695857677411|2.8860074541721437|0.2757500164117377|-3.919680955819602|
| stddev|  53.5094673816375|0.2817926773467503| 0.446898758463185|15.560721063721333|
|    min|             -53.0|2.3729120029701067|                 0|             -48.0|
|    max|            2028.0| 3.330210784571528|                 1|             229.0|
+-------+------------------+------------------+------------------+------------------+



In [66]:
bd4.describe('ARR_DELAY','LogD','DELAY','RETARD').toPandas()

,summary,ARR_DELAY,LogD,DELAY,RETARD
0,count,30466,30466,30466,30466
1,mean,13.300695857677411,2.8860074541721437,0.2757500164117377,-3.919680955819602
2,stddev,53.5094673816375,0.2817926773467503,0.446898758463185,15.560721063721333
3,min,-53.0,2.3729120029701067,0,-48.0
4,max,2028.0,3.330210784571528,1,229.0


In [67]:
sqlContext.registerDataFrameAsTable(bd4, "bd4")

bd5 = sqlContext.sql("select *, case \
               when CRS_DEP_TIME <= 800 then 1 \
               when 800 < CRS_DEP_TIME and CRS_DEP_TIME <= 1200 then 2 \
               when 1200 < CRS_DEP_TIME and CRS_DEP_TIME <= 1600 then 3 \
               when 1600 < CRS_DEP_TIME and CRS_DEP_TIME <= 2100 then 4 \
               else 1 end as Horario \
               from bd4")

In [68]:
bd5.select('Horario').describe().show()

+-------+-----------------+
|summary|          Horario|
+-------+-----------------+
|  count|            30466|
|   mean|2.539322523468785|
| stddev|1.136901609153115|
|    min|                1|
|    max|                4|
+-------+-----------------+



In [69]:
bd5.select('Horario').groupBy('Horario').count().show()

+-------+-----+
|Horario|count|
+-------+-----+
|      1| 7519|
|      2| 7382|
|      3| 7180|
|      4| 8385|
+-------+-----+



## Exportar o banco de dados para HDF

In [70]:
bd5.write.format(
    "com.databricks.spark.csv"
).option(
    "header", "true"
).mode('overwrite').save('/user/cloudera/spark/voos/bd5.csv') 